In [3]:
import torch
import wandb
import datasets 
import numpy as np
from tqdm import tqdm
import torch.nn as nn 
import torch.nn.functional as F
import matplotlib.pyplot as plt 
from torch.utils.data.dataset import Dataset
from transformers import PreTrainedTokenizerFast
from transformers import RobertaForSequenceClassification
from sklearn.metrics import classification_report, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import os 
import warnings
from os import PathLike

api = wandb.Api()
artifact = api.artifact('henry-williams/opcode-malberta/model-rusreqta:v1', type='model')
artifact_dir = artifact.download()

warnings.filterwarnings("ignore")

device = torch.device('mps')
dataset = datasets.load_from_disk('../data/raw')
tokenizer = PreTrainedTokenizerFast.from_pretrained(artifact_dir)
model = RobertaForSequenceClassification.from_pretrained(artifact_dir).to(device)

BATCH_SIZE = 256

dataset['test'] = dataset['test']

/Users/henrywilliams/Documents/programming/python/ai/malbert-test/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Downloading large artifact model-rusreqta:v1, 87.83MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.7


In [ ]:
def make_preds(sample):
    input = tokenizer(
        sample,
        padding='max_length',
        max_length=32,
        return_overflowing_tokens=True,
        truncation=True,
        return_special_tokens_mask=True,
    )
    input_ids = torch.tensor(input['input_ids'])
    attention_mask = torch.tensor(input['attention_mask'])
    full_logits = []

    for ids, mask in zip(input_ids.split(BATCH_SIZE), attention_mask.split(BATCH_SIZE)):
        torch.mps.empty_cache()
        with torch.no_grad():
            logits = model(ids.to(device), mask.to(device)).logits
        full_logits.append(logits)

    logits = torch.vstack(full_logits)
    return logits

def make_ds():
    model.eval()

    predictions = []
    actuals = []

    for i, sample in enumerate(tqdm(dataset['test'], position=0)):
        logits = make_preds(sample['text']).unsqueeze(0)
        
        predictions.append(logits)
        actuals.append(sample['label'])
    return predictions, actuals 

In [ ]:
class PredictionsDataset(Dataset):
    def __init__(self, preds, labels):
        if len(preds) != len(labels):
            raise ValueError("Mismatch in size between x and y")

        self.preds = preds 
        self.labels = labels 
    
    def __len__(self):
        return len(self.preds)

    def __getitem__(self, index):
        return self.preds[index], self.labels[index]

if not os.path.exists('logits-ds.pt'):
    predictions, actuals = make_ds()
    d = PredictionsDataset(predictions, actuals)
    torch.save(d, 'logits-ds.pt')
else: 
    d = torch.load('logits-ds.pt', weights_only=False)


In [ ]:
class Reducer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Reducer, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size 
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, 2)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        z = self.fc(out[:, -1, :])
        return z

In [ ]:
r = Reducer(2, 32, 3)
r = r.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(r.parameters())

loss_history = []

for logits, label in tqdm(d):
    torch.mps.empty_cache()
    actual = F.one_hot(torch.tensor(torch.tensor(label)), 2).unsqueeze(0).to(torch.float32).to(device)
    
    optimizer.zero_grad()
    predicted = r(logits)
    loss = criterion(predicted, actual)
    loss_history.append(loss)
    loss.backward() 
    
    optimizer.step()

In [ ]:
plt.plot(torch.tensor(loss_history))

In [15]:
ids = tokenizer.encode(dataset['test'][0]['text'], add_special_tokens=True)

tokenizer.decode(ids, skip_special_tokens=False)

'ret nop nop nop nop test add nop add add sub sub in sub mov mov add in mov add xor leave mov add add add mov mov add out mov add mov add add add mov mov add jmp mov add mov add add add mov mov add lodsb mov add mov add add add mov mov add mov mov add cmp cmp cmp pop pop jne cmovs movsxd push cmp add add rol cmp cmp push add add je imul mov mov add jb mov add mov or lodsb out add mov add test sal je mov mov add add call or jg add call cmpsd test add mov mov adc adc mov add mov adc mov adc call xchg test add mov mov adc loopne mov add mov adc mov adc call cmpsd adc add mov mov add movabs mov add cmp cmp add je push xor ror add add sub ret nop add add mov add add call jle add jmp mov nop add add movzx mov push sbb cmp cmp cli or add je cmp cmp cmp cli or add jne test cmp mov test add jbe xchg js dec mov nop clc add xor leave test ror setne xchg shr jmp dec nop add add lea lea or and adc add call in sbb add xor ror add add sub ret nop add add cmp js je jbe xchg cmp inc mov mov sub call ad